# MAST30034 Project 1
## Statistical Modelling

In [302]:
import pandas as pd

LaGuardia Airport was chosen as the area taken for weather measurements as out of the 3 major airports in NYC, it is the most centrally located and thus will be the best estimate of the overal weather across the city.

In [303]:
LG18 = pd.read_csv("/Users/jackmelleuish/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Jack/Uni/2022/Semester 2/Data Science/Project 1/Weather Data/LaGuardia2018.csv")
LG19 = pd.read_csv("/Users/jackmelleuish/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Jack/Uni/2022/Semester 2/Data Science/Project 1/Weather Data/LaGuardia2019.csv")
print(len(LG18.columns))
print(len(LG19.columns))

100
102


The datasets have approx 100 columns with highly specific weather for a variety of needs, as this project is looking at how weather and other variables effect tipping in NYC, the variables chosen were the main distinct variables commonly reported on: Wind Direction, Wind Speed, Temperature, Dew Point, Sea Level Pressure

In [304]:
LG18 = LG18[['DATE', 'WND', 'TMP', 'DEW', 'SLP']]
LG19 = LG19[['DATE', 'WND', 'TMP', 'DEW', 'SLP']]

Cleaning up the data

In [305]:
import numpy as np

LG18['date']=LG18['DATE'].str.extract(r'(\d{4}-\d{2}-\d{2})')
LG18['wind_speed']=LG18['WND'].str.extract(r'\d{3},\d,\w,(\d{4},\d)')
LG18['wind_direction']=LG18['WND'].str.extract(r'(\d{3},\d),\w,\d{4},\d')
LG18 = LG18.rename(columns={"TMP":"temp", "DEW":"dew_point", "SLP":"pressure"})
LG18['wind_direction'] = LG18['wind_direction'].str.replace(',','.',regex=True)
LG18['wind_speed'] = LG18['wind_speed'].str.replace(',','.',regex=True)
LG18['temp'] = LG18['temp'].str.replace(',','.',regex=True)
#LG18['temp'] = LG18['temp'].str.replace('0','',regex=True)
LG18['dew_point'] = LG18['dew_point'].str.replace(',','.',regex=True)
LG18['pressure'] = LG18['pressure'].str.replace(',','.',regex=True)
LG18 = LG18.drop(['DATE', 'WND'], axis =1)

LG19['date']=LG19['DATE'].str.extract(r'(\d{4}-\d{2}-\d{2})')
LG19['wind_speed']=LG19['WND'].str.extract(r'\d{3},\d,\w,(\d{4},\d)')
LG19['wind_direction']=LG19['WND'].str.extract(r'(\d{3},\d),\w,\d{4},\d')
LG19 = LG19.rename(columns={"TMP":"temp", "DEW":"dew_point", "SLP":"pressure"})
LG19['wind_direction'] = LG19['wind_direction'].str.replace(',','.',regex=True)
LG19['wind_speed'] = LG19['wind_speed'].str.replace(',','.',regex=True)
LG19['temp'] = LG19['temp'].str.replace(',','.',regex=True)
LG19['dew_point'] = LG19['dew_point'].str.replace(',','.',regex=True)
LG19['pressure'] = LG19['pressure'].str.replace(',','.',regex=True)
LG19 = LG19.drop(['DATE', 'WND'], axis =1)


Converting Strings to Floats:

In [306]:
LG18 = LG18.drop([5320, 7718, 8555, 8970, 13060])
LG18['pressure'] = pd.to_numeric(LG18['pressure'], downcast='float')
LG18['temp'] = pd.to_numeric(LG18['temp'], downcast='float')
LG18['dew_point'] = pd.to_numeric(LG18['dew_point'], downcast='float')
LG18['wind_speed'] = pd.to_numeric(LG18['wind_speed'], downcast='float')
LG18['wind_direction'] = pd.to_numeric(LG18['wind_direction'], downcast='float')

LG19 = LG19.drop([2410, 3700, 3862, 8355, 8544, 10130])
LG19 = LG19.drop(LG19.index[10120:10157])
LG19['pressure'] = pd.to_numeric(LG19['pressure'], downcast='float')
LG19['temp'] = pd.to_numeric(LG19['temp'], downcast='float')
LG19['dew_point'] = pd.to_numeric(LG19['dew_point'], downcast='float')
LG19['wind_speed'] = pd.to_numeric(LG19['wind_speed'], downcast='float')
LG19['wind_direction'] = pd.to_numeric(LG19['wind_direction'], downcast='float')


### Invalidating data:
#### Temp
Maximum recorded temperature in NYC was 106F and the Mminium was -15F, thus these were set as the bounds

In [307]:
LG18 = LG18.drop(LG18[LG18.temp > 106].index)
LG18 = LG18.drop(LG18[LG18.temp < -15].index)
LG19 = LG19.drop(LG19[LG19.temp > 106].index)
LG19 = LG19.drop(LG19[LG19.temp < -15].index)

#### Dew Point
Maximum recorded dew point in NYC was 81F and the Mminium was -49F, thus these were set as the bounds

In [308]:
LG18 = LG18.drop(LG18[LG18.dew_point > 81].index)
LG18 = LG18.drop(LG18[LG18.dew_point < -49].index)
LG19 = LG19.drop(LG19[LG19.dew_point > 81].index)
LG19 = LG19.drop(LG19[LG19.dew_point < -49].index)

#### Pressure
There isnt much data for the max and min sea level pressure of NYC, so the upper bounds were set to be the worlds max and min to be safe.

In [309]:
LG18 = LG18.drop(LG18[LG18.pressure > 10600].index)
LG18 = LG18.drop(LG18[LG18.pressure < 8700].index)
LG19 = LG19.drop(LG19[LG19.pressure > 10600].index)
LG19 = LG19.drop(LG19[LG19.pressure < 8700].index)

#### Wind Speed 
highest ever record was 185km/h during hurricane sandy  and ofcourse the slowest is set to 0.

In [310]:
LG18 = LG18.drop(LG18[LG18.wind_speed > 185].index)
LG18 = LG18.drop(LG18[LG18.wind_speed < 0].index)
LG19 = LG19.drop(LG19[LG19.wind_speed > 185].index)
LG19 = LG19.drop(LG19[LG19.wind_speed < 0].index)

#### Wind Direction 
wind direction is set in relation to compass degrees and thus the max is 360 and min is 0:

In [311]:
LG18 = LG18.drop(LG18[LG18.wind_direction > 185].index)
LG18 = LG18.drop(LG18[LG18.wind_direction < 0].index)
LG19 = LG19.drop(LG19[LG19.wind_direction > 185].index)
LG19 = LG19.drop(LG19[LG19.wind_direction < 0].index)
LG18 = LG18.reset_index()
LG19 = LG19.reset_index()